In [4]:
import os
import requests
import gradio as gr
from datetime import datetime

from openai import OpenAI
from dotenv import load_dotenv

In [5]:
load_dotenv()

AVALAI_API_KEY = os.getenv("AVALAI_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
GEOAPIFY_API_KEY = os.getenv("GEOAPIFY_API_KEY")
AVIATIONSTACK_API_KEY = os.getenv("AVIATIONSTACK_API_KEY")

client = OpenAI(
    api_key=AVALAI_API_KEY,
    base_url="https://api.avalai.ir/v1"
)

In [ ]:
class TravelMemory:
    def __init__(self):
        self.preferences = None
        self.budget = None
        self.destination = None

In [7]:
def call_llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a professional AI travel assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

In [8]:
def get_weather(destination):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": destination,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric"
    }

    r = requests.get(url, params=params)
    data = r.json()

    temp = data["main"]["temp"]
    condition = data["weather"][0]["description"]
    wind = data["wind"]["speed"]

    return {
        "temp": temp,
        "condition": condition,
        "wind": wind
    }

def weather_alert(destination):
    weather = get_weather(destination)

    prompt = f"""
    Explain this weather data for a traveler:
    Temperature: {weather['temp']}°C
    Condition: {weather['condition']}
    Wind speed: {weather['wind']} m/s
    """
    return call_llm(prompt)

In [9]:
def get_attractions(destination):
    url = "https://api.geoapify.com/v2/places"
    params = {
        "categories": "tourism.attraction",
        "filter": f"place:{destination}",
        "limit": 5,
        "apiKey": GEOAPIFY_API_KEY
    }

    r = requests.get(url, params=params)
    data = r.json()

    places = []
    for item in data.get("features", []):
        places.append(item["properties"].get("name", "Unknown place"))

    return places

def attractions_summary(destination):
    places = get_attractions(destination)

    prompt = f"""
    Create a short attractions list for a traveler visiting {destination}.
    Places: {places}
    """
    return call_llm(prompt)

In [10]:
def get_flight_status(destination):
    url = "http://api.aviationstack.com/v1/flights"
    params = {
        "access_key": AVIATIONSTACK_API_KEY,
        "arr_iata": destination[:3].upper(),
        "limit": 1
    }

    r = requests.get(url, params=params)
    data = r.json()

    if not data.get("data"):
        return "No recent flight data found."

    flight = data["data"][0]
    return f"""
Flight {flight['flight']['iata']} 
Status: {flight['flight_status']}
Departure: {flight['departure']['airport']}
Arrival: {flight['arrival']['airport']}
"""

def generate_itinerary(destination, preferences, attractions):
    prompt = f"""
    Create a 3-day itinerary for {destination}.

    Preferences: {preferences}
    Attractions: {attractions}

    Make it realistic and traveler-friendly.
    """
    return call_llm(prompt)

In [11]:
class AdvancedTravelAgent:
    def __init__(self):
        self.memory = TravelMemory()

    def set_preferences(self, preferences, budget):
        self.memory.preferences = preferences
        self.memory.budget = budget

        prompt = f"""
        Suggest 3 destinations based on:
        Preferences: {preferences}
        Budget: {budget}
        """
        return call_llm(prompt)

    def finalize_trip(self, destination):
        self.memory.destination = destination

        weather = weather_alert(destination)
        attractions = attractions_summary(destination)
        flights = get_flight_status(destination)

        itinerary = generate_itinerary(
            destination,
            self.memory.preferences,
            attractions
        )

        alerts = f"""
🌦 Weather:
{weather}

✈️ Flights:
{flights}
        """

        return itinerary, alerts

In [12]:
agent = AdvancedTravelAgent()

with gr.Blocks(title="AI Travel Assistant") as demo:
    gr.Markdown("## ✈️ AI Travel Assistant (Real Data Agent)")

    with gr.Tab("Step 1: Preferences"):
        pref = gr.Textbox(label="Preferences", placeholder="beaches, food, culture")
        budget = gr.Textbox(label="Budget", placeholder="$1500")
        btn1 = gr.Button("Suggest Destinations")
        out1 = gr.Textbox(label="Suggestions", lines=8)

        btn1.click(agent.set_preferences, [pref, budget], out1)

    with gr.Tab("Step 2: Final Plan"):
        dest = gr.Textbox(label="Chosen Destination")
        btn2 = gr.Button("Generate Plan")

        itinerary = gr.Textbox(label="Itinerary", lines=12)
        alerts = gr.Textbox(label="Real-time Alerts", lines=10)

        btn2.click(agent.finalize_trip, dest, [itinerary, alerts])

demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
